In [1]:

descriptorlength=256

In [6]:
cv2.__version__

'4.1.2'

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==4.4.0.40 --force-reinstall

# auth code

# 4/1AY0e-g5osE0pIkc3ub9eYD4d_7JGUhqJEYCrLbgXJkyhzdn00K3uxNfvtxY

     |████████████████████████████████| 55.6MB 75kB/s 
  Using cached https://files.pythonhosted.org/packages/14/32/d3fa649ad7ec0b82737b92fefd3c4dd376b0bb23730715124569f38f3a08/numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: opencv-contrib-python 3.4.2.17
    Uninstalling opencv-contrib-python-3.4.2.17:
      Successfully uninstalled opencv-contrib-python-3.4.2.17


In [5]:
import cv2 as cv
import numpy as np
import pandas as pd
#import util
import os
import random
import glob
import cv2
import PIL
from PIL import Image
def FrameCapture(path,skipfactor=3): 
    #clearing extracted frames
    files = glob.glob('extractedframes/*')
    for f in files:
        os.remove(f)
        
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print(fps)
    vidObj.set(cv2.CAP_PROP_FPS, 30)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print(fps)
    # Used as counter variable 
    count = 0
    ind = 0
  
    # checks whether frames were extracted 
    success = 1
    listOfImages = []
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read()
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Saves the frames with frame-count 
        if(count%skipfactor==0):
            listOfImages.append(image)
            
            #cv2.imwrite("extractedframes/frame%d.jpg" %ind, image)
            #cv2.imshow('frame%d.jpg'%ind,image)
            
            ind += 1
        count += 1
        type(image)
    return listOfImages

In [ ]:
def ListGrayscaleFrameArrays(listOfFrames):
    listOfFrameArray=[]
    listOfFrames.pop()
    
    for ind,frame  in enumerate(listOfFrames):
        #converts RBG image to Gray scale and returns as numpy 2D array
        ff= frame.copy()
        gray = cv2.cvtColor(ff, cv2.COLOR_BGR2GRAY)
        
#         print(gray.shape)
#         scale_percent = 60 # percent of original size
#         width = int(1000)
#         height = int(1000)
#         dim = (width, height)
#         # resize image
#         resized = cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)
        cv2.imwrite("grayframes/frame%d.jpg" %ind, gray)
        #using PIL
        #grayscalearray = np.array(Image.fromarray(frame).convert('L'))
        grayscalearray = np.array(gray)
        
        listOfFrameArray.append(grayscalearray)
        
    return listOfFrameArray



In [ ]:

def StoreFrames(path,skipfactor=2):
    #extracted features stores all gray frames of a  video individually for further processing
    #clear extracted frames folder first
    files = glob.glob('extractedframes/*')
    for f in files:
        os.remove(f)
    FrameCapture(path,skipfactor)




In [ ]:
#A  video is converted to a numpy array
def ConvertVideoToGrayscaleArray(path):
    listofImages = FrameCapture(path,2)

    GrayScaleList= ListGrayscaleFrameArrays(listofImages)
    
    ArrayofGrayScaleArray = np.array(GrayScaleList)
    
    #print(ArrayofGrayScaleArray.shape)
    return ArrayofGrayScaleArray

In [ ]:
files = glob.glob('siftoutput/*')
cnt=0
for f in files:
  cnt=cnt+1
print(cnt)
  

0


In [ ]:
#movementfiltering

#stores binarized list of frames in jpg format 
def BinarizeGrayscaleArray(ArrayofGrayScaleArray,path,threshold=150):
    files = glob.glob('binaryframes/*')
    for f in files:
        os.remove(f)
    for i in range(1,ArrayofGrayScaleArray.shape[0]):
        binaryImage=abs(ArrayofGrayScaleArray[i]-ArrayofGrayScaleArray[i-1])
        data = Image.fromarray(binaryImage)
        data.save('temporalderivatives/'+'frame%d.jpg'%i)
        #Binarization
        binaryImage[binaryImage > threshold] = 255
        binaryImage[binaryImage <= threshold] = 0 
        
        data = Image.fromarray(binaryImage) 
        data.save(path+'/'+'frame%d.jpg'%i)
  

In [8]:
cd /content/drive/MyDrive/ViolenceDetection

/content/drive/MyDrive/ViolenceDetection


In [9]:
pwd

'/content/drive/MyDrive/ViolenceDetection'

In [ ]:
def erodeBinaryFrames(binaryframespath,erodedframespath):
    erosionkernel = np.array([
        [0,1,1,0],
        [0,1,1,0],
        [1,1,1,1],
        [0,1,1,0],
        
        
        
    ],np.uint8)
    
#     [0,0,0,1,1,0,0,0],
#     [0,0,1,1,1,1,0,0],
#     [0,1,1,1,1,1,1,0],
#     [1,1,1,1,1,1,1,1],
#     [1,1,1,1,1,1,1,1],
#     [0,1,1,1,1,1,1,0],
#     [0,0,1,1,1,1,0,0],
#     [0,0,0,1,1,0,0,0]
    dilationkernel = np.array([
        [0,0,0,1,1,0,0,0],
        [0,0,1,1,1,1,0,0],
        [0,1,1,1,1,1,1,0],
        [1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1],
        [0,1,1,1,1,1,1,0],
        [0,0,1,1,1,1,0,0],
        [0,0,0,1,1,0,0,0]
    ],np.uint8)
    

    # Taking a matrix of size 5 as the kernel 
    kernel = np.ones((3,3), np.uint8)
    kernel[0][0]=0
    kernel[0][2]=0
    kernel[2][0]=0
    kernel[2][2]=0
    
    kernel1 = np.ones((2,2), np.uint8)
    
    elipticalkernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    
    # The first parameter is the original image, 
    # kernel is the matrix with which image is  
    # convolved and third parameter is the number  
    # of iterations, which will determine how much  
    # you want to erode/dilate a given image.
    files = glob.glob(erodedframespath+'/*')
    for f in files:
        os.remove(f)
    files = glob.glob(binaryframespath+'/*')
    ind=0
    for f in files:
        img = cv2.imread(f, 0) 
        #img = cv2.blur(img,(5,5))
        img = cv2.erode(img,elipticalkernel, iterations=2)
        #img = cv2.dilate(img,kernel, iterations=4)
        #img = cv2.erode(img, elipticalkernel, iterations=5) 
        img = cv2.dilate(img, elipticalkernel, iterations=2)
        img = cv2.erode(img, kernel, iterations=5) 
        #img = cv2.dilate(img, kernel, iterations=1)
        #img = cv2.erode(img, erosionkernel, iterations=1) 
        
        #img = cv2.morphologyEx(img, cv2.MORPH_OPEN, elipticalkernel, iterations=3) 
        
        cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        #print(len(cnts[0]))
        #print(len(cnts[0]),ind+1)
    
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 20:
                cv2.drawContours(img, [c], -1, (0,0,0), -1)
        cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(len(cnts),'after removal')
        if len(cnts[0])>5:
            cv2.imwrite(erodedframespath+"/frame%d.jpg" %ind,img)
            ind=ind+1




In [ ]:
def preprocess(videopath):
    #A  video is converted to a numpy array
    ArrayofGrayScaleArray= ConvertVideoToGrayscaleArray(videopath)
    BinarizeGrayscaleArray(ArrayofGrayScaleArray,'binaryframes',150)
    erodeBinaryFrames("binaryframes","morphologicallyopenedframes")
#upto morphological opening
#preprocess('/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi95.avi')
#preprocess('samplefights/fi1_xvid.avi')

#preprocess('samplefights/50.mpg')

In [ ]:
import math

cells = [15, 31, 47, 63, 79, 95, 111, 127, 143, 159, 175, 191, 207, 223, 239, 255]


def gen_neighbors(x, y):
    neighbor_1 = [[x - 8, y + 8], [x - 7, y + 8], [x - 6, y + 8], [x - 5, y + 8],
          [x - 8, y + 7], [x - 7, y + 7], [x - 6, y + 7], [x - 5, y + 7],
          [x - 8, y + 6], [x - 7, y + 6], [x - 6, y + 6], [x - 5, y + 6],
          [x - 8, y + 5], [x - 7, y + 5], [x - 6, y + 5], [x - 5, y + 5]]

    neighbor_2 = [[x - 4, y + 8], [x - 3, y + 8], [x - 2, y + 8], [x - 1, y + 8],
          [x - 4, y + 7], [x - 3, y + 7], [x - 2, y + 7], [x - 1, y + 7],
          [x - 4, y + 6], [x - 3, y + 6], [x - 2, y + 6], [x - 1, y + 6],
          [x - 4, y + 5], [x - 3, y + 5], [x - 2, y + 5], [x - 1, y + 5]]

    neighbor_3 = [[x + 1, y + 8], [x + 2, y + 8], [x + 3, y + 8], [x + 4, y + 8],
          [x + 1, y + 7], [x + 2, y + 7], [x + 3, y + 7], [x + 4, y + 7],
          [x + 1, y + 6], [x + 2, y + 6], [x + 3, y + 6], [x + 4, y + 6],
          [x + 1, y + 5], [x + 2, y + 5], [x + 3, y + 5], [x +  4, y + 5]]

    neighbor_4 = [[x + 5, y + 8], [x + 6, y + 8], [x + 7, y + 8], [x + 8, y + 8],
          [x + 5, y + 7], [x + 6, y + 7], [x + 7, y + 7], [x + 8, y + 7],
          [x + 5, y + 6], [x + 6, y + 6], [x + 7, y + 6], [x + 8, y + 6],
          [x + 5, y + 5], [x + 6, y + 5], [x + 7, y + 5], [x + 8, y + 5]]

    neighbor_5 = [[x - 8, y + 4], [x - 7, y + 4], [x - 6, y + 4], [x - 5, y + 4],
          [x - 8, y + 3], [x - 7, y + 3], [x - 6, y + 3], [x - 5, y + 3],
          [x - 8, y + 2], [x - 7, y + 2], [x - 6, y + 2], [x - 5, y + 2],
          [x - 8, y + 1], [x - 7, y + 1], [x - 6, y + 1], [x - 5, y + 1]]

    neighbor_6 = [[x - 4, y + 4], [x - 3, y + 4], [x - 2, y + 4], [x - 1, y + 4],
          [x - 4, y + 3], [x - 3, y + 3], [x - 2, y + 3], [x - 1, y + 3],
          [x - 4, y + 2], [x - 3, y + 2], [x - 2, y + 2], [x - 1, y + 2],
          [x - 4, y + 1], [x - 3, y + 1], [x - 2, y + 1], [x - 1, y + 1]]

    neighbor_7 = [[x + 1, y + 4], [x + 2, y + 4], [x + 3, y + 4], [x + 4, y + 4],
          [x + 1, y + 3], [x + 2, y + 3], [x + 3, y + 3], [x + 4, y + 3],
          [x + 1, y + 2], [x + 2, y + 2], [x + 3, y + 2], [x + 4, y + 2],
          [x + 1, y + 1], [x + 2, y + 1], [x + 3, y + 1], [x + 4, y + 1]]

    neighbor_8 = [[x + 5, y + 4], [x + 6, y + 4], [x + 7, y + 4], [x + 8, y + 4],
          [x + 5, y + 3], [x + 6, y + 3], [x + 7, y + 3], [x + 8, y + 3],
          [x + 5, y + 2], [x + 6, y + 2], [x + 7, y + 2], [x + 8, y + 2],
          [x + 5, y + 1], [x + 6, y + 1], [x + 7, y + 1], [x + 8, y + 1]]

    neighbor_9 = [[x - 8, y - 4], [x - 7, y - 4], [x - 6, y - 4], [x - 5, y - 4],
          [x - 8, y - 3], [x - 7, y - 3], [x - 6, y - 3], [x - 5, y - 3],
          [x - 8, y - 2], [x - 7, y - 2], [x - 6, y - 2], [x - 5, y - 2],
          [x - 8, y - 1], [x - 7, y - 1], [x - 6, y - 1], [x - 5, y - 1]]

    neighbor_10 = [[x - 4, y - 4], [x - 3, y - 4], [x - 2, y - 4], [x - 1, y - 4],
           [x - 4, y - 3], [x - 3, y - 3], [x - 2, y - 3], [x - 1, y - 3],
           [x - 4, y - 2], [x - 3, y - 2], [x - 2, y - 2], [x - 1, y - 2],
           [x - 4, y - 1], [x - 3, y - 1], [x - 2, y - 1], [x - 1, y - 1]]

    neighbor_11 = [[x + 1, y - 4], [x + 2, y - 4], [x + 3, y - 4], [x + 4, y - 4],
           [x + 1, y - 3], [x + 2, y - 3], [x + 3, y - 3], [x + 4, y - 3],
           [x + 1, y - 2], [x + 2, y - 2], [x + 3, y - 2], [x + 4, y - 2],
           [x + 1, y - 1], [x + 2, y - 1], [x + 3, y - 1], [x + 4, y - 1]]

    neighbor_12 = [[x + 5, y - 4], [x + 6, y - 4], [x + 7, y - 4], [x + 8, y - 4],
           [x + 5, y - 3], [x + 6, y - 3], [x + 7, y - 3], [x + 8, y - 3],
           [x + 5, y - 2], [x + 6, y - 2], [x + 7, y - 2], [x + 8, y - 2],
           [x + 5, y - 1], [x + 6, y - 1], [x + 7, y - 1], [x + 8, y - 1]]

    neighbor_13 = [[x - 8, y - 8], [x - 7, y - 8], [x - 6, y - 8], [x - 5, y - 8],
           [x - 8, y - 7], [x - 7, y - 7], [x - 6, y - 7], [x - 5, y - 7],
           [x - 8, y - 6], [x - 7, y - 6], [x - 6, y - 6], [x - 5, y - 6],
           [x - 8, y - 5], [x - 7, y - 5], [x - 6, y - 5], [x - 5, y - 5]]

    neighbor_14 = [[x - 4, y - 8], [x - 3, y - 8], [x - 2, y - 8], [x - 1, y - 8],
           [x - 4, y - 7], [x - 3, y - 7], [x - 2, y - 7], [x - 1, y - 7],
           [x - 4, y - 6], [x - 3, y - 6], [x - 2, y - 6], [x - 1, y - 6],
           [x - 4, y - 5], [x - 3, y - 5], [x - 2, y - 5], [x - 1, y - 5]]

    neighbor_15 = [[x + 1, y - 8], [x + 2, y - 8], [x + 3, y - 8], [x + 4, y - 8],
           [x + 1, y - 7], [x + 2, y - 7], [x + 3, y - 7], [x + 4, y - 7],
           [x + 1, y - 6], [x + 2, y - 6], [x + 3, y - 6], [x + 4, y - 6],
           [x + 1, y - 5], [x + 2, y - 5], [x + 3, y - 5], [x + 4, y - 5]]

    neighbor_16 = [[x + 5, y - 8], [x + 6, y - 8], [x + 7, y - 8], [x + 8, y - 8],
           [x + 5, y - 7], [x + 6, y - 7], [x + 7, y - 7], [x + 8, y - 7],
           [x + 5, y - 6], [x + 6, y - 6], [x + 7, y - 6], [x + 8, y - 6],
           [x + 5, y - 5], [x + 6, y - 5], [x + 7, y - 5], [x + 8, y - 5]]

    neighbors = neighbor_1+neighbor_2+neighbor_3+neighbor_4+neighbor_5+neighbor_6+neighbor_7+neighbor_8+ \
                neighbor_9+neighbor_10+neighbor_11+neighbor_12+neighbor_13+neighbor_14+neighbor_15+neighbor_16

    return neighbors

def gen_arc_histogram(direction, histogram):
    if direction > (2*math.pi)/3 and direction <= math.pi/3:
        histogram[0]+=1
    elif direction > math.pi/3 and direction <= math.pi/6:
        histogram[1]+=1
    elif direction > math.pi/6 and direction <= (11*math.pi)/6:
        histogram[2]+=1
    elif direction > (11*math.pi)/6 and direction <= (5*math.pi)/3:
        histogram[3]+=1
    elif direction > (5*math.pi)/3 and direction <= (4*math.pi)/3:
        histogram[4]+=1
    elif direction > (4*math.pi)/3 and direction <= (5*math.pi)/4:
        histogram[5]+=1
    elif direction > (5*math.pi)/4 and direction <= math.pi:
        histogram[6]+=1
    elif direction > math.pi and direction <= (2*math.pi)/3:
        histogram[7]+=1
    return histogram

In [ ]:
#SIFT

import cv2 as cv
import numpy as np
import pandas as pd
#import util
import os
import random

# params for ShimTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# params for Lucas Kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))


def capture_frame(video_path, i):
    cap = cv.VideoCapture(video_path)
    cap.set(1, i)
    ret, frame = cap.read()

    return ret, frame


def count_frames(video_path):
    cap = cv.VideoCapture(video_path)
    frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    return frames


def gen_sift_features(frame):
    sift = cv.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(frame, None)

    return keypoints, descriptors


def keypoints_to_coordinates(keypoints):
    keypoints_xy = []

    for kp in keypoints:
        keypoints_xy.append([kp.pt[0], kp.pt[1]])

    return keypoints_xy


def has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd):
    sm_keypoints_xy = []
    sm_keypoints = []
    sm_descriptors = []

    for i in range(len(keypoints)):
        distance_x = keypoints_xy[i].T[0] - p1[i].T[0]
        distance_y = keypoints_xy[i].T[1] - p1[i].T[1]

        if distance_x > lambd or distance_y > lambd:
            sm_keypoints_xy.append([int(keypoints_xy[i].T[0]), int(keypoints_xy[i].T[1])])
            sm_keypoints.append(keypoints[i])
            sm_descriptors.append(descriptors[i])

    return sm_keypoints_xy, sm_keypoints, sm_descriptors


def gen_hof(x, y, frame, next_frame):
    hof = []
    histogram = [0, 0, 0, 0, 0, 0, 0, 0]
    neighbors = gen_neighbors(x, y)
    neighbors = np.float32(np.array(neighbors)[:, np.newaxis, :])
    p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, neighbors, None, **lk_params)

    for i in range(len(p1)):
        direction = np.arctan(p1[i].T[0]/p1[i].T[1])
        histogram = gen_arc_histogram(direction, histogram)
        if i in cells:
            hof += histogram
            histogram = [0, 0, 0, 0, 0, 0, 0, 0]

    return hof




In [ ]:
# def gen_mosift_features(video_path, lambd, interval, sample_size):
#     frames = count_frames(video_path)
#     mosift_descriptors = []

#     for i in range(1, frames-1, interval):
#         #try:
#         ret, frame = capture_frame(video_path, i)
#         ret, next_frame = capture_frame(video_path, i+1)
#         print(type(frame))
#         keypoints, descriptors = gen_sift_features(frame)
#         keypoints_xy = keypoints_to_coordinates(keypoints)
#         keypoints_xy = np.float32(np.array(keypoints_xy)[:, np.newaxis, :])
#         p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, keypoints_xy, None, **lk_params)
#         sm_keypoints_xy, sm_keypoints, sm_descriptors = has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd)

#         for j in range(len(sm_keypoints)):
#             hof = np.array(gen_hof(sm_keypoints_xy[j][0], sm_keypoints_xy[j][1], frame, next_frame))
#             mosift_descriptors.append(list(np.concatenate((sm_descriptors[j], hof))))
#         #except:
#             #print('Exception in '+video_path+', frame '+str(i))

#     random_mosift_descriptors = random.sample(mosift_descriptors, k=int(len(mosift_descriptors)*sample_size))

#     return random_mosift_descriptors


# def run_feature_extractor(input_path, output_path, lambd, interval, sample_size):
#     listing = os.listdir(input_path)
#     progress_count = 0
#     for video_name in listing:
#         progress_count += 1
#         print("# progress: "+str(progress_count)+'/'+str(len(listing)))
#         video_path = input_path+'/'+ video_name
        
#         df_mosift_features = pd.DataFrame(gen_mosift_features(video_path, lambd, interval, sample_size))
#         print(df_mosift_features.shape)
#         #df_mosift_features.to_csv(output_path+'/'+ video_name[:-4]+".csv", mode='a', header=False, index=False)

# run_feature_extractor('siftvideo','siftoutput',0.9,1,1)

In [ ]:
def gen_mosift_features(morphopath, lambd, interval, sample_size):
    files = glob.glob(morphopath+'/*')
    frames=0
    for f in files:
        frames = frames+1
    mosift_descriptors = []

    for i in range(1, frames-1, interval):
        #try:
        j=i+1
        print(i)
        #frame = cv2.imread(morphopath+'/frame%d'%i,cv2.IMREAD_UNCHANGED)
        frame = cv.imread(morphopath+'/frame%d.jpg'%i,1)
        frame=cv.normalize(frame, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        next_frame = cv.imread(morphopath+'/frame%d.jpg'%j,1)
        next_frame=cv.normalize(next_frame, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = gen_sift_features(frame)
        keypoints_xy = keypoints_to_coordinates(keypoints)
        #print(keypoints_xy)
        l=len(keypoints_xy)
#         print(l)
        keypoints_xy = np.float32(np.array(keypoints_xy)[:, np.newaxis, :])
#         keypoints_xy = np.float32(np.array(keypoints_xy))
#         keypoints_xy.reshape(l,1,2)
        #print(keypoints_xy.shape)
        p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, keypoints_xy, None, **lk_params)
        sm_keypoints_xy, sm_keypoints, sm_descriptors = has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd)

        for j in range(len(sm_keypoints)):
            hof = np.array(gen_hof(sm_keypoints_xy[j][0], sm_keypoints_xy[j][1], frame, next_frame))
            mosift_descriptors.append(list(np.concatenate((sm_descriptors[j], hof))))
        #except:
            #print('Exception in '+video_path+', frame '+str(i))

    random_mosift_descriptors = random.sample(mosift_descriptors, k=int(len(mosift_descriptors)*sample_size))

    return random_mosift_descriptors


def run_feature_extractor(morphopath, output_path,video_name, lambd, interval, sample_size):
    
    df_mosift_features = pd.DataFrame(gen_mosift_features(morphopath, lambd, interval, sample_size))
    print(df_mosift_features)
    df_mosift_features.to_csv(output_path+'/'+video_name[70:-4]+".csv", mode='a', header=False, index=False)

#run_feature_extractor('morphologicallyopenedframes','siftoutput',video_name,0.85,1,1)

In [ ]:
ar=np.zeros((1,descriptorlength))
#ar.reshape(1,256)
print(ar.shape)
zerof=pd.DataFrame(ar)

print(zerof.shape)


(1, 256)
(1, 256)


In [ ]:
#final upto extracting descriptors and stroing them in "siftoutput" folder as .csv files.
def storeDescriptors(videopath):
    videos = glob.glob(videopath+'/*')
    skippedcount=0
    for video in videos:
        #video=video[13:]
        print(video)
        csvname=video[70:-4]
        print(csvname)
        features = glob.glob('/content/drive/MyDrive/ViolenceDetection/siftoutput/*')
        flag=0
        for featurefile in features:
          #print(featurefile,"  ",features )
          if csvname == featurefile[52:-4]:
            flag=1
            skippedcount=skippedcount+1
            print(featurefile[52:-4] , "skipped ==>  ",skippedcount)
            break
        if flag==0:
          preprocess(video)
          run_feature_extractor('morphologicallyopenedframes','siftoutput',video,0.85,1,1)


In [ ]:
from sklearn.cluster import KMeans
def bagOfVisualWords(descriptorspath,noofclusters):
    descriptors = glob.glob(descriptorspath+'/*')
    maindataframe=pd.DataFrame()
    for cnt,descriptor in enumerate(descriptors):
        
        try:
          df=pd.read_csv(descriptor,header=None)
        except:
          ar=np.zeros((1,descriptorlength))
          df=pd.DataFrame(ar)
          
        maindataframe=maindataframe.append(df)
        print(df.shape,"  ==>  ", descriptor,cnt+1)
        
    print(maindataframe.shape)
    
    kmeans = KMeans(n_clusters = noofclusters).fit(maindataframe)
    
    return kmeans
           




In [ ]:
            
numberofclusters=100 
def featureExtractor(descriptorspath,numberofclusters):
    descriptors = glob.glob(descriptorspath+'/*')
    trainfeatures=np.empty(shape=[0, 100])
    for descriptor in descriptors:
        
        try:
          df=pd.read_csv(descriptor,header=None)
        except:
          ar=np.zeros((1,descriptorlength))
          df=pd.DataFrame(ar)
        print(df.shape)
        histogram=kmeans.predict(df)
        
        featurevector=np.zeros(numberofclusters,)
        for i in range(0,len(histogram)):
          featurevector[histogram[i]]=featurevector[histogram[i]]+1
        #print(featurevector)
        trainfeatures=np.vstack([trainfeatures, featurevector])    
    
    return trainfeatures


    
    

    


In [ ]:
storeDescriptors('/content/drive/MyDrive/movies DATASET Violence Detection/samplefights')


/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi7.avi
newfi7
newfi7 skipped ==>   1
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi8.avi
newfi8
newfi8 skipped ==>   2
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi3.avi
newfi3
newfi3 skipped ==>   3
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi6.avi
newfi6
newfi6 skipped ==>   4
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi68.avi
newfi68
newfi68 skipped ==>   5
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi69.avi
newfi69
newfi69 skipped ==>   6
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi4.avi
newfi4
newfi4 skipped ==>   7
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi5.avi
newfi5
newfi5 skipped ==>   8
/content/drive/MyDrive/movies DATASET Violence Detection/samplefights/newfi9.avi
newfi9
newfi9 skipped ==>

In [ ]:
#storeDescriptors('/content/drive/MyDrive/ViolenceDetection/samplefights')

In [ ]:
storeDescriptors('/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig')

/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/31.mpg
31
31 skipped ==>   1
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/80.mpg
80
80 skipped ==>   2
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/70.mpg
70
70 skipped ==>   3
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/34.mpg
34
34 skipped ==>   4
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/72.mpg
72
72 skipped ==>   5
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/73.mpg
73
73 skipped ==>   6
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/74.mpg
74
74 skipped ==>   7
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/79.mpg
79
79 skipped ==>   8
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/36.mpg
36
36 skipped ==>   9
/content/drive/MyDrive/movies DATASET Violence Detection/samplenonfig/39.mpg
39
39 skipped ==>   10
/content/

In [ ]:
videos = glob.glob('siftoutput/*')
for video in videos:
    print(video)
        

siftoutput/newfi7.csv
siftoutput/newfi8.csv
siftoutput/newfi3.csv
siftoutput/newfi6.csv
siftoutput/newfi68.csv
siftoutput/newfi69.csv
siftoutput/newfi4.csv
siftoutput/newfi5.csv
siftoutput/newfi9.csv
siftoutput/newfi2.csv
siftoutput/newfi1.csv
siftoutput/newfi100.csv
siftoutput/newfi31.csv
siftoutput/newfi76.csv
siftoutput/newfi37.csv
siftoutput/newfi38.csv
siftoutput/newfi73.csv
siftoutput/newfi35.csv
siftoutput/newfi82.csv
siftoutput/newfi75.csv
siftoutput/newfi39.csv
siftoutput/newfi36.csv
siftoutput/newfi71.csv
siftoutput/newfi33.csv
siftoutput/newfi46.csv
siftoutput/newfi81.csv
siftoutput/newfi30.csv
siftoutput/newfi32.csv
siftoutput/newfi78.csv
siftoutput/newfi44.csv
siftoutput/newfi72.csv
siftoutput/newfi34.csv
siftoutput/newfi79.csv
siftoutput/newfi43.csv
siftoutput/newfi40.csv
siftoutput/newfi70.csv
siftoutput/newfi77.csv
siftoutput/newfi45.csv
siftoutput/newfi41.csv
siftoutput/newfi80.csv
siftoutput/newfi42.csv
siftoutput/newfi84.csv
siftoutput/newfi49.csv
siftoutput/newfi50.

In [ ]:
kmeans=bagOfVisualWords('siftoutput',100)

KeyboardInterrupt: ignored

In [ ]:
trainset=featureExtractor('siftoutput',100)
print(trainset.shape)

(9673, 256)
(10206, 256)
(3929, 256)
(6311, 256)
(3862, 256)
(834, 256)
(3798, 256)
(6211, 256)
(7019, 256)
(2644, 256)
(2644, 256)
(3968, 256)
(11766, 256)
(10138, 256)
(9199, 256)
(2420, 256)
(1372, 256)
(10233, 256)
(3705, 256)
(6382, 256)
(5682, 256)
(3006, 256)
(4334, 256)
(8742, 256)
(15891, 256)
(7316, 256)
(9497, 256)
(8685, 256)
(4150, 256)
(6420, 256)
(1372, 256)
(10731, 256)
(5122, 256)
(8859, 256)
(926, 256)
(2704, 256)
(5277, 256)
(10122, 256)
(2912, 256)
(7765, 256)
(9907, 256)
(5692, 256)
(10187, 256)
(10440, 256)
(3213, 256)
(2397, 256)
(8551, 256)
(7260, 256)
(3660, 256)
(4143, 256)
(4949, 256)
(6532, 256)
(892, 256)
(4469, 256)
(4462, 256)
(4162, 256)
(9152, 256)
(3313, 256)
(9108, 256)
(7261, 256)
(7270, 256)
(3637, 256)
(1005, 256)
(2767, 256)
(3220, 256)
(262, 256)
(466, 256)
(377, 256)
(646, 256)
(913, 256)
(267, 256)
(1149, 256)
(494, 256)
(203, 256)
(431, 256)
(779, 256)
(654, 256)
(292, 256)
(389, 256)
(850, 256)
(3005, 256)
(887, 256)
(2338, 256)
(6108, 256)
(

In [ ]:
print(kmeans)

NameError: ignored

In [10]:
#saving trainset
import pickle
#pickle.dump(trainset, open("trainfeatures.pkl", "wb"))
trainset=pickle.load(open("trainfeatures.pkl","rb"))

In [11]:
# Its important to use binary mode 
#knnPickle = open('knnpickle_file', 'wb') 

# source, destination 
#pickle.dump(kmeans, knnPickle)                      


# load the model from disk
kmeans = pickle.load(open('knnpickle_file', 'rb'))

In [12]:
print(trainset)

[[ 81.  77. 127. ...  96. 104.  95.]
 [ 85.  84. 109. ... 115.  94. 110.]
 [ 25.  32.  22. ...  45.  30.  28.]
 ...
 [  5.  10.   0. ...   3.   1.   5.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [14]:
def filltargetvariable():
    csvnames = glob.glob('siftoutput/*')
    cnt=0
    for names in csvnames:
      cnt=cnt+1
    target=np.zeros(cnt)
    csvnames = glob.glob('siftoutput/*')
    cnt=0
    for ind,names in enumerate(csvnames):
      if names[11]=='n':
        target[ind]=1
      print(ind,names,target[ind])
      

    # nonfightcount=0
    # for video in videos:
    #     nonfightcount=nonfightcount+1
    # videos = glob.glob('samplefights/*')
    # fightcount=0
    # for video in videos:
    #     fightcount=fightcount+1
    # target1=np.zeros(nonfightcount)

    # target2=np.ones(fightcount)
    # target= np.append(target1,target2)
    return target

target=filltargetvariable()    

0 siftoutput/newfi7.csv 1.0
1 siftoutput/newfi8.csv 1.0
2 siftoutput/newfi3.csv 1.0
3 siftoutput/newfi6.csv 1.0
4 siftoutput/newfi68.csv 1.0
5 siftoutput/newfi69.csv 1.0
6 siftoutput/newfi4.csv 1.0
7 siftoutput/newfi5.csv 1.0
8 siftoutput/newfi9.csv 1.0
9 siftoutput/newfi2.csv 1.0
10 siftoutput/newfi1.csv 1.0
11 siftoutput/newfi100.csv 1.0
12 siftoutput/newfi31.csv 1.0
13 siftoutput/newfi76.csv 1.0
14 siftoutput/newfi37.csv 1.0
15 siftoutput/newfi38.csv 1.0
16 siftoutput/newfi73.csv 1.0
17 siftoutput/newfi35.csv 1.0
18 siftoutput/newfi82.csv 1.0
19 siftoutput/newfi75.csv 1.0
20 siftoutput/newfi39.csv 1.0
21 siftoutput/newfi36.csv 1.0
22 siftoutput/newfi71.csv 1.0
23 siftoutput/newfi33.csv 1.0
24 siftoutput/newfi46.csv 1.0
25 siftoutput/newfi81.csv 1.0
26 siftoutput/newfi30.csv 1.0
27 siftoutput/newfi32.csv 1.0
28 siftoutput/newfi78.csv 1.0
29 siftoutput/newfi44.csv 1.0
30 siftoutput/newfi72.csv 1.0
31 siftoutput/newfi34.csv 1.0
32 siftoutput/newfi79.csv 1.0
33 siftoutput/newfi43.csv 1.

In [ ]:
pip install xgboost

**XGBoost**

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
# check accuracy score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(trainset, target, test_size = 0.3, random_state = 3)

params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 20,
            'learning_rate': 0.2,
            'n_estimators':20
        }
xgb_clf = XGBClassifier(**params)
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)



print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)*100),'%')

XGBClassifier(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=20, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

XGBoost model accuracy score: 95.0820 %


**Support Vector Machine**

In [ ]:
from sklearn import svm
# check accuracy score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(trainset, target, test_size = 0.3, random_state = 3)
clf = svm.SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)



print('SVM model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)*100),'%')


SVM model accuracy score: 91.8033 %


In [ ]:
print(y_pred-y_test)


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [ ]:
print(y_test)

[1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.]


**AdaBoost**

In [34]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# check accuracy score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(trainset, target, test_size = 0.3, random_state = 2)

In [36]:
abc = AdaBoostClassifier(n_estimators=40,
                         learning_rate=0.1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)

print('Adaboost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)*100),'%')

Adaboost model accuracy score: 88.5246 %


**Random Forest**

In [44]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(trainset, target, test_size = 0.3, random_state = 2)
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Random forest model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)*100),'%')

Random forest model accuracy score: 91.8033 %
